In [3]:
from dataset.star import STAR
# data loader
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
import json


In [2]:
import torch
from mplug_owl_video.modeling_mplug_owl import MplugOwlForConditionalGeneration
from transformers import AutoTokenizer
from mplug_owl_video.processing_mplug_owl import MplugOwlImageProcessor, MplugOwlProcessor

pretrained_ckpt = 'MAGAer13/mplug-owl-llama-7b-video'
# model = MplugOwlForConditionalGeneration.from_pretrained(
#     pretrained_ckpt,
#     torch_dtype=torch.float16,
# )
model = MplugOwlForConditionalGeneration.from_pretrained( pretrained_ckpt, device_map={'': 0}, torch_dtype=torch.bfloat16)

image_processor = MplugOwlImageProcessor.from_pretrained(pretrained_ckpt)
tokenizer = AutoTokenizer.from_pretrained(pretrained_ckpt)
processor = MplugOwlProcessor(image_processor, tokenizer)

print('Model loaded.')
# We use a human/AI template to organize the context as a multi-turn conversation.
# <|video|> denotes an video placehold.
# prompts = [
# '''The following is a conversation between a curious human and AI assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.
# Human: <|video|>
# Human: Which object was put down by the person? 0. "The food., 1. "The laptop.", 2. "The book.", 3. "The pillow."
# AI: ''']


# video_list = ['YSKX3.mp4']

# # generate kwargs (the same in transformers) can be passed in the do_generate()
# generate_kwargs = {
#     'do_sample': True,
#     'top_k': 5,
#     'max_length': 512
# }
# print('Start generating...')
# inputs = processor(text=prompts, videos=video_list, num_frames=4, return_tensors='pt')
# # inputs = {k: v.bfloat16() if v.dtype == torch.float else v for k, v in inputs.items()}
# inputs = {k: v.to(model.device) for k, v in inputs.items()}
# with torch.no_grad():
#     res = model.generate(**inputs, **generate_kwargs)
# sentence = tokenizer.decode(res.tolist()[0], skip_special_tokens=True)
# print(sentence)

/home/vann/anaconda3/envs/mplug_owl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


install flash-attn first.
Model loaded.


In [4]:
# read csv into question_id,video_id,Keyframe_IDs
import csv
import ast
qid_list = []
vid_list = []
kfid_list = []
# Open the CSV file
with open('Video_Keyframe_IDs.csv', 'r') as csvfile:
    # Create a CSV reader object
    csvreader = csv.DictReader(csvfile)

    # Iterate through each row in the CSV file
    for row in csvreader:
        # row variable is a dictionary that represents a row in csv
        qid_list.append(row['question_id'])
        vid_list.append(row['video_id'])
        kfid_list.append(row['Keyframe_IDs'])

### testing with single video


In [63]:
# We use a human/AI template to organize the context as a multi-turn conversation.
# <|video|> denotes an video placehold.
# prompts = [
# '''
# Which object was put down by the person? 0. "The food., 1. "The laptop.", 2. "The book.", 3. "The pillow.".
# The answer is ''',

# '''
# Which object was put down by the person? 0. "The clothes., 1. "The pillow.", 2. "The towel.", 3. "The laptop.".
# The answer is ''',
# ]

prompts = [
'''
Human: <|video|>
Human: What is this video about?
AI: ''',
]
# '''
# Human: <|video|>
# Human: What is this video about?
# AI: '''

# ]

video_list = ["videos/" + vid_list[0]+'.mp4']
key_frame_list = [ast.literal_eval(kfid_list[0])]
# video_list = ['YSKX3.mp4', 'E9UYZ.mp4']

# generate kwargs (the same in transformers) can be passed in the do_generate()
generate_kwargs = {
    'do_sample': False,
    'top_k': 3,
    'max_length': 300
}


In [64]:
inputs = processor(text=prompts, videos=video_list, kfid = key_frame_list,num_frames=10, return_tensors='pt')
inputs = {k: v.bfloat16() if v.dtype == torch.float else v for k, v in inputs.items()}
inputs = {k: v.to(model.device) for k, v in inputs.items()}
model.eval()
with torch.no_grad():
    res = model.generate(**inputs, **generate_kwargs)
for i in res.tolist():
    sentence = tokenizer.decode(i, skip_special_tokens=True)
    print(sentence)
    
# sentence = tokenizer.decode(res.tolist()[0], skip_special_tokens=True)
# print(sentence)

The video shows a young man, possibly a boy, playing with a bed sheet on a bed in a room. He is jumping and having fun, creating a playful atmosphere. The room is well-lit and has a few chairs and a table nearby. The bed is situated in the center of the room, with the bed sheet hanging from the bed frame.


### iterate all test data

In [57]:
test_data = STAR("test")
data_lo1der = DataLoader(test_data, batch_size=4, shuffle=False, num_workers=3)

generate_kwargs = {
    'do_sample': False,
    'top_k': 3,
    'max_length': 8
}


Num test data: 7377


In [98]:

model.eval()
answers =[]
# for id, data in tqdm(enumerate(test_data), total=len(test_data)):
for id, data in enumerate(test_data):
    if id > 20:
        break
    text, qid, answer = data
    qid_index = qid_list.index(qid)
    video_list = ["videos/" + vid_list[qid_index]+'.mp4']
    key_frame_list = [ast.literal_eval(kfid_list[qid_index])]
    question = text['q_text'].split(':')[1].split("\n")[0]
    #remove first space
    question = question[1:]
    option = text['o_text'].split(':')[1]
    choices = text['options']
    ans = text['a_text'].split(':')[1]
    
#     prompts = [
#     f'''The following is a conversation between a human and AI assistant. The assistant gives an accurate answer to the user's question.
# Human: <|video|>
# Human: {question},  0. {choices[0]}, 1. {choices[1]}, 2. {choices[2]}, 3. {choices[3]}, choose between these options.
# AI: The answer is ''']

    prompts = [
    f'''The following is a conversation between a human and AI assistant. The assistant gives accurate answers to the user's question.
Human: <|video|>
Human: {question}, 
AI: The answer is '''
]



    
    inputs = processor(text=prompts, videos=video_list, kfid = key_frame_list,num_frames=12, return_tensors='pt')
    
    inputs = {k: v.bfloat16() if v.dtype == torch.float else v for k, v in inputs.items()}
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        res = model.generate(**inputs, **generate_kwargs)
    print(prompts[0])
    for k in res.tolist():
        sentence = tokenizer.decode(k, skip_special_tokens=True)
        answers.append(sentence)
        print("AI: " + sentence)

with open('answers.json', 'w') as f:
    json.dump(answers, f)

The following is a conversation between a human and AI assistant. The assistant gives accurate answers to the user's question.
Human: <|video|>
Human: Which object was put down by the person?, 
AI: The answer is 
AI: a bed sheet.
The following is a conversation between a human and AI assistant. The assistant gives accurate answers to the user's question.
Human: <|video|>
Human: Which object was taken by the person?, 
AI: The answer is 
AI: a pillow.
The following is a conversation between a human and AI assistant. The assistant gives accurate answers to the user's question.
Human: <|video|>
Human: Which object was thrown by the person?, 
AI: The answer is 
AI: a pillow.
The following is a conversation between a human and AI assistant. The assistant gives accurate answers to the user's question.
Human: <|video|>
Human: Which object was tidied up by the person?, 
AI: The answer is 
AI: a bed.
The following is a conversation between a human and AI assistant. The assistant gives accurate a

### generate output

In [95]:
answer_options = [int(i[0]) for i in answers]

In [96]:
answer_options

[3, 3, 3, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 0, 3, 3, 2, 0, 2]

In [ ]:
answer_qid_list = []
for id, data in tqdm(enumerate(test_data), total=len(test_data)):
    text, qid, answer = data
    answer_qid_list.append(qid)

In [187]:
final_answers_dict = {"Interaction":[], "Sequence":[], "Prediction" : [], "Feasibility": []}
for id, answer_qid in enumerate(answer_qid_list):
    if "Interaction" in answer_qid:
        final_answers_dict["Interaction"].append({"question_id":answer_qid, "answer":int(answer_options[id])})
    elif "Sequence" in answer_qid:
        final_answers_dict["Sequence"].append({"question_id":answer_qid, "answer":int(answer_options[id])})
    elif "Prediction" in answer_qid:
        final_answers_dict["Prediction"].append({"question_id":answer_qid, "answer":int(answer_options[id])})
    elif "Feasibility" in answer_qid:
        final_answers_dict["Feasibility"].append({"question_id":answer_qid, "answer":int(answer_options[id])})

In [188]:
#dump to json
with open('final_answers.json', 'w') as f:
    json.dump(final_answers_dict, f)
